In [ ]:
pip install fastparquet

In [4]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Nov-2022/preds_concat_gzip.parquet'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

# ## Reading data-files
# train = pd.read_parquet(file_content_stream_1)
# train.head()

In [5]:
file_content_stream_1

In [10]:
df = pd.read_parquet('s3://analytics-data-science-competitions/Tabular-Playground-Series/Tabular-Playground-Nov-2022/preds_concat_gzip.parquet', engine = 'fastparquet')

In [11]:
df.head()

,0.7301891713.csv,0.6750726968.csv,0.7194704070.csv,0.7107007521.csv,0.6952032365.csv,0.7311830751.csv,0.6736005999.csv,0.7205109360.csv,0.7298116981.csv,0.6737624943.csv,0.7105097012.csv,0.7163693161.csv,0.7225567087.csv,0.6872126167.csv,0.7292947020.csv,0.6798503735.csv,0.7130672940.csv,0.6709092823.csv,0.7391105353.csv,0.6616334858.csv,0.6897190987.csv,0.6963558296.csv,0.7170726761.csv,0.7156669797.csv,0.6814483512.csv,0.7167410246.csv,0.6981892492.csv,0.7119194656.csv,0.6932545022.csv,0.6593521643.csv,0.6743157708.csv,0.7125579799.csv,0.7069356577.csv,0.6884713711.csv,0.6702128703.csv,0.6564147356.csv,0.6977737011.csv,0.6638768547.csv,0.7032756348.csv,0.6862982027.csv,0.6864219834.csv,0.7088904051.csv,0.7212180410.csv,0.7421980731.csv,0.6825341422.csv,0.6848737762.csv,0.7144978161.csv,0.6658856476.csv,0.7195078907.csv,0.7096861940.csv,...,0.6870791547.csv,0.6967096004.csv,0.7060967827.csv,0.7113625862.csv,0.6694947166.csv,0.6900693308.csv,0.7128591265.csv,0.6818718747.csv,0.6858034178.csv,0.7122377012.csv,0.7144335192.csv,0.7083170570.csv,0.7142499638.csv,0.7136242881.csv,0.6888742488.csv,0.7087145574.csv,0.6566530931.csv,0.7197358544.csv,0.7101817035.csv,0.7520219713.csv,0.7215952204.csv,0.6795805487.csv,0.7083706040.csv,0.6941218921.csv,0.6926641668.csv,0.6869857003.csv,0.7198785912.csv,0.7287761089.csv,0.7035686386.csv,0.7336496744.csv,0.6872533788.csv,0.7005346672.csv,0.7369069391.csv,0.7156537235.csv,0.7121002711.csv,0.7289228406.csv,0.7174789519.csv,0.7032181627.csv,0.6668147232.csv,0.7345858839.csv,0.7056743178.csv,0.6850481924.csv,0.7359858591.csv,0.7068431477.csv,0.7121179915.csv,0.6613117872.csv,0.6843637618.csv,0.6781392004.csv,0.7222809303.csv,target
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.713797,0.894976,0.817926,0.810555,0.791920,0.831945,0.774879,0.741634,0.756136,0.727903,0.763435,0.769848,0.802273,0.760738,0.839669,0.60,0.767607,0.823816,0.872288,0.655031,0.736038,0.798806,0.778560,0.782497,0.812633,0.809022,0.800860,0.805305,0.741648,0.971786,0.72789,0.814,0.774592,0.787105,0.915710,0.880978,0.778571,0.844169,0.784163,0.782036,0.780643,0.721790,0.698241,0.808,0.798784,0.477387,0.772133,0.905410,0.773577,0.726302,...,0.763574,0.752004,0.776569,0.798669,0.807704,0.767779,0.831448,0.792231,0.763278,0.779712,0.716066,0.703275,0.768230,0.804498,0.786796,0.718659,0.662422,0.813924,0.743625,0.663370,0.790927,0.775915,0.778482,0.774662,0.777772,0.778873,0.775081,0.824640,0.817052,0.830458,0.78967,0.786102,0.758304,0.783439,0.796190,0.756559,0.797150,0.761263,0.924655,0.716121,0.810299,0.768360,0.796567,0.838916,0.765676,0.659398,0.799853,0.831508,0.824168,0.0
1,0.663330,0.640218,0.574168,0.574927,0.667912,0.756107,0.638533,0.605146,0.624973,0.605908,0.555053,0.565853,0.825874,0.746966,0.803360,0.76,0.715148,0.712462,0.752800,0.544701,0.755066,0.654167,0.594217,0.592934,0.757777,0.624227,0.635633,0.557927,0.501230,0.692161,0.76032,0.614,0.638995,0.622001,0.551986,0.666823,0.680000,0.689739,0.635571,0.651588,0.645543,0.610949,0.603778,0.829,0.820298,0.503128,0.582307,0.649761,0.696104,0.570501,...,0.580294,0.618501,0.570683,0.600717,0.662596,0.587837,0.597760,0.777449,0.672334,0.536803,0.635282,0.564045,0.591958,0.692422,0.630778,0.600586,0.511741,0.599708,0.593361,0.721298,0.571635,0.644542,0.601430,0.681132,0.593403,0.631658,0.656799,0.566144,0.618821,0.635893,0.62927,0.744298,0.561998,0.580583,0.611598,0.589977,0.550247,0.547826,0.713663,0.644158,0.640244,0.646700,0.700815,0.633100,0.657584,0.536717,0.649205,0.601636,0.532060,1.0
2,0.772586,0.816572,0.880483,0.829270,0.765237,0.821705,0.839576,0.824883,0.841343,0.781148,0.843974,0.829139,0.407582,0.762720,0.816622,0.76,0.876730,0.754158,0.851190,0.714048,0.813104,0.802805,0.860778,0.844159,0.773301,0.798508,0.793820,0.834659,0.642200,0.816399,0.80876,0.750,0.823076,0.795760,0.817964,0.802251,0.764286,0.809974,0.829432,0.805009,0.798551,0.830638,0.841210,0.831,0.769674,0.637722,0.867044,0.818138,